In [57]:
import random
from datetime import datetime
from tqdm import tqdm
import numpy as np
from numpy.linalg import norm
from scipy.linalg import expm, logm

random.seed(42)

### Экспонента кососимметрической матрицы

Коссосиметрическая матрица A 3 x 3 имеет вид.

$$
\begin{pmatrix}
0 & -с & b\\
c & 0 & -a\\
-b & a & 0
\end{pmatrix}
$$

Положим $\theta = \sqrt{a^2 + b^2 + c^2}$

Тогда матричную экспоненту можно найти по формуле [Родригеса](https://en.wikipedia.org/wiki/Axis%E2%80%93angle_representation#Exponential_map_from_so(3)_to_SO(3)):

$$e^A = I_{3} + \frac{\sin{\theta}}{\theta}A + \frac{1-cos{\theta}}{\theta^2}A^2$$

Возьмем для примера, a = 0.5, b = 0.75, c = 1. То есть будем рассматривать матрицу:

$$
\begin{pmatrix}
0 & -1 & 0.75\\
1 & 0 & -0.5\\
-0.75 & 0.5 & 0
\end{pmatrix}
$$

In [58]:
mat = np.array([[0, -1, 0.75], [1, 0, -0.5], [-0.75, 0.5, 0]])

mat

array([[ 0.  , -1.  ,  0.75],
       [ 1.  ,  0.  , -0.5 ],
       [-0.75,  0.5 ,  0.  ]])

Сначала воспользуемся встроенной функцией в ```Scipy```. В дальнейшем будем считать ее эталонной и сравнивать с ней кастомную реализацию.

In [59]:
mat_exp_scipy = expm(mat)

mat_exp_scipy

array([[ 0.32984822, -0.56330438,  0.75755418],
       [ 0.88497723,  0.46387857, -0.04039755],
       [-0.32865704,  0.68374326,  0.65152107]])

Напишем кастомную функцию на основе формулы Родригеса.

In [ ]:
def custom_expm_3d(a, b, c):
    mat = np.array([[0, -c, b], [c, 0, -a], [-b, a, 0]])
    theta = norm([a, b, c])
    mat_exp = np.eye(3) + (np.sin(theta) / theta) * mat + ((1 - np.cos(theta)) / theta**2) * (mat @ mat)
    return mat_exp

In [ ]:
custom_expm_3d(0.5, 0.75, 1)

array([[ 0.32984822, -0.56330438,  0.75755418],
       [ 0.88497723,  0.46387857, -0.04039755],
       [-0.32865704,  0.68374326,  0.65152107]])

Сравним скорость работы `scipy` реализации и нашей на 1_000_000 сгенерированных матриц.

In [ ]:
synthetic_data_3d = [[np.random.random(), np.random.random(), np.random.random()] for _ in range(1_000_000)]

In [ ]:
scipy_mat_exp_3d_list = []

start_time = datetime.now()
for triplet in tqdm(synthetic_data_3d):
    a, b, c = triplet
    mat = np.array([[0, -c, b], [c, 0, -a], [-b, a, 0]])
    scipy_mat_exp_3d_list.append(expm(mat))
finish_time = (datetime.now() - start_time).total_seconds()

print(f'Время для Scipy реализации: {round(finish_time, 2)} секунд. ')

100%|██████████| 1000000/1000000 [00:33<00:00, 29718.67it/s]

Время для Scipy реализации: 33.65 секунд. 


In [ ]:
custom_mat_exp_3d_list = []

start_time = datetime.now()
for triplet in tqdm(synthetic_data_3d):
    a, b, c = triplet
    custom_mat_exp_3d_list.append(custom_expm_3d(a, b, c))
finish_time = (datetime.now() - start_time).total_seconds()

print(f'Время для кастомной реализации: {round(finish_time, 2)} секунд. ')

100%|██████████| 1000000/1000000 [00:06<00:00, 157882.81it/s]

Время для кастомной реализации: 6.34 секунд. 


In [ ]:
max_exp_diff = []

for scipy_mat_exp, custom_mat_exp in zip(scipy_mat_exp_3d_list, custom_mat_exp_3d_list):
    max_exp_diff.append(norm(scipy_mat_exp - custom_mat_exp))

print(f'L2 норма разности матричных экспонент, полученных Scipy и кастомной реализациями {round(np.mean(max_exp_diff), 5)}')

L2 норма разности матричных экспонент, полученных Scipy и кастомной реализациями 0.0


Формула работает идеально

### Логарифм от матричной экспоненты

Пусть $R$ - матрица вращения (экспонента кососимметрической матрицы $A$), тогда 

$$A = \log{R} = \frac{\theta}{2\sin{\theta}}(R-R^T)$$

где $tr(R) = 1 + 2 \cos{\theta}$

напишем функцию для вычисления логорифма матрицы вращения

In [ ]:
def custom_logm_3d(r):
    theta = np.arccos((np.trace(r) - 1) / 2)
    mat_log = theta / (2 * np.sin(theta)) * (mat_exp_scipy - mat_exp_scipy.transpose())
    return mat_log

In [62]:
mat

array([[ 0.  , -1.  ,  0.75],
       [ 1.  ,  0.  , -0.5 ],
       [-0.75,  0.5 ,  0.  ]])

In [ ]:
custom_logm_3d(custom_expm_3d(0.5, 0.75, 1))

array([[ 0.  , -1.  ,  0.75],
       [ 1.  ,  0.  , -0.5 ],
       [-0.75,  0.5 ,  0.  ]])

Получили исходную матрицу. Причем `Scipy` logm отрабатывает некорректно, поэтому кастомное решение надежнее.

In [67]:
logm(expm(mat))

array([[-2.59502092e-16, -1.00000000e+00,  7.50000000e-01],
       [ 1.00000000e+00, -2.44084791e-16, -5.00000000e-01],
       [-7.50000000e-01,  5.00000000e-01, -2.03852358e-16]])